In [ ]:
!pip install pybamm
import pybamm as pb

In [ ]:
import os
os.chdir(pybamm.__path__[0] + "/..")

# initialise the model
model = pb.BaseModel()

In [ ]:
# dimensional parameters
k = pb.Parameter("Reaction rate constant [m.s-1]")
L_0 = pb.Parameter("Initial thickness [m]")
V_hat = pb.Parameter("Partial molar volume [m3.mol-1]")
c_inf = pb.Parameter("Bulk electrolyte solvent concentration [mol.m-3]")


def D(cc):
    return pb.FunctionParameter(
        "Diffusivity [m2.s-1]", {"Solvent concentration [mol.m-3]": cc}
    )


# define dimensionless parameters in the model
xi = pb.SpatialVariable("xi", domain="SEI layer", coord_sys="cartesian")
c = pb.Variable("Solvent concentration [mol.m-3]", domain="SEI layer")
L = pb.Variable("SEI thickness [m]")

In [ ]:
# governing equations

# SEI reaction flux
R = k * pb.BoundaryValue(c, "left")

# solvent concentration equation
N = -1 / L * D(c) * pb.grad(c)
dcdt = (V_hat * R) / L * pb.inner(xi, pb.grad(c)) - 1 / L * pb.div(N)

# SEI thickness equation
dLdt = V_hat * R

# add equations to dictionary
# keys = variables to be solved, values = RHS of governing equations for each variable
model.rhs = {c: dcdt, L: dLdt}

In [ ]:
# Boundary conditions
D_left = pb.BoundaryValue(
    D(c), "left"
)  # pb requires BoundaryValue(D(c)) and not D(BoundaryValue(c))
grad_c_left = R * L / D_left

c_right = c_inf

model.boundary_conditions = {
    c: {"left": (grad_c_left, "Neumann"), "right": (c_right, "Dirichlet")}
}

In [ ]:
# set initial conditions
c_init = c_inf
L_init = L_0
model.initial_conditions = {c: c_init, L: L_init}

In [ ]:
# output variables
model.variables = {
    "SEI thickness [m]": L,
    "SEI growth rate [m]": dLdt,
    "Solvent concentration [mol.m-3]": c,
}

In [ ]:
# define geometry
geometry = pb.Geometry(
    {"SEI layer": {xi: {"min": pb.Scalar(0), "max": pb.Scalar(1)}}}
)


def Diffusivity(cc):
    return cc * 10 ** (-12)


# parameter values (not physically based, for example only!)
param = pb.ParameterValues(
    {
        "Reaction rate constant [m.s-1]": 1e-6,
        "Initial thickness [m]": 1e-6,
        "Partial molar volume [m3.mol-1]": 10,
        "Bulk electrolyte solvent concentration [mol.m-3]": 1,
        "Diffusivity [m2.s-1]": Diffusivity,
    }
)

# process model and geometry
param.process_model(model)
param.process_geometry(geometry)

# mesh and discretise
submesh_types = {"SEI layer": pb.Uniform1DSubMesh}
var_pts = {xi: 100}
mesh = pb.Mesh(geometry, submesh_types, var_pts)

spatial_methods = {"SEI layer": pb.FiniteVolume()}
disc = pb.Discretisation(mesh, spatial_methods)
disc.process_model(model)

In [ ]:
# solve
solver = pb.ScipySolver()
t = [0, 100]  # solve for 100s
solution = solver.solve(model, t)

# post-process output variables
L_out = solution["SEI thickness [m]"]
c_out = solution["Solvent concentration [mol.m-3]"]

In [ ]:
# plotting the results
import matplotlib.pyplot as plt
import numpy as np

# plot SEI thickness in microns as a function of t in microseconds
# and concentration in mol/m3 as a function of x in microns
L_0_eval = param.evaluate(L_0)
xi = np.linspace(0, 1, 100)  # dimensionless space


def plot(t):
    _, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    ax1.plot(solution.t, L_out(solution.t) * 1e6)
    ax1.plot(t, L_out(t) * 1e6, "r.")
    ax1.set_ylabel(r"SEI thickness [$\mu$m]")
    ax1.set_xlabel(r"t [s]")

    ax2.plot(xi * L_out(t) * 1e6, c_out(t, xi))
    ax2.set_ylim(0, 1.1)
    ax2.set_xlim(0, L_out(solution.t[-1]) * 1e6)
    ax2.set_ylabel("Solvent concentration [mol.m-3]")
    ax2.set_xlabel(r"x [$\mu$m]")

    plt.tight_layout()
    plt.show()


import ipywidgets as widgets

widgets.interact(
    plot, t=widgets.FloatSlider(min=0, max=solution.t[-1], step=0.1, value=0)
);